In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\naura\\Desktop\\end-to-end-ML\\computer-vision\\Bone-fracture-xray-classification'

In [2]:
# Entity 

from pydantic import BaseModel
from pathlib import Path

class ModelTrainingConfig(BaseModel):
    root_dir: Path
    trained_model_path: Path
    custom_trained_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
# configuration manager 

from boneFractureClassification.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from boneFractureClassification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_training_config(self) -> ModelTrainingConfig:
        
        training = self.config.model_training
        base_model = self.config.base_model
        params = self.params
        
        training_data = os.path.join(self.config.data_ingestion.local_data_file, "Bone_Fracture_Binary_Classification/train")
        validation_data = os.path.join(self.config.data_ingestion.local_data_file,"Bone_Fracture_Binary_Classification/val")
        
        create_directories([
            Path(training.root_dir)
        ])
        
        model_training_config = ModelTrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.model_training_path),
            custom_trained_model_path = Path(base_model.custom_trained_model_path),
            training_data = Path(training_data),
            validation_data = Path(validation_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        
        return model_training_config
        
        

In [5]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True


In [6]:
checkpoint = ModelCheckpoint(filepath="artifacts/training/custom_train_model.h5",
                             monitor="val_loss",
                             mode="auto",
                             save_best_only=True)
early_stopping = EarlyStopping(verbose=1, patience=3)

In [7]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
        
    def load_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.custom_trained_model_path
        )
        
    def train_val_data_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.validation_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
    
    @staticmethod    
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train_model(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks=[checkpoint, early_stopping]
        )
        
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [10]:
# pipeline
#tf.debugging.set_log_device_placement(True)
try:
    
    config = ConfigurationManager()
    training_config = config.get_model_training_config()
    train = ModelTraining(config=training_config)
    with tf.device('/device:GPU:0'):
        train.load_base_model()
        train.train_val_data_generator()
        train.train_model()
except RuntimeError as re:
    raise re
except Exception as e:
    raise e

[2024-06-02 01:21:20,115: INFO: common: yaml file: config\config.yaml loaded successfully!]
[2024-06-02 01:21:20,119: INFO: common: yaml file: params.yaml loaded successfully!]
[2024-06-02 01:21:20,123: INFO: common: created directory at: artifacts]
[2024-06-02 01:21:20,127: INFO: common: created directory at: artifacts\training]
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignV